<a href="https://colab.research.google.com/github/M-Ati-git/Colab-works/blob/main/Real_Life_Bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

def fetch_budget(imdb_id: str) -> int:,
    \"\"\"Fetch the 'Budget' from Box Office Mojo using imdb_id.,
,
    :param imdb_id: The ID of the movie (e.g., \"tt0424880\").,
    :return:        Budget as an integer (e.g., 61000000), or None if not found.,
    \"\"\",
    url = f\"https://www.boxofficemojo.com/title/{imdb_id}/\",
    response = requests.get(url),
    soup = BeautifulSoup(response.content, 'html.parser'),
    budget_section = soup.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile'),
,
    if budget_section:,
        for div in budget_section.find_all('div', class_='a-section a-spacing-none'):,
            if 'Budget' in div.text:,
                try:,
                    budget = div.find_all('span')[1].text.strip(),
                    return int(budget.replace(\"$\", \"\").replace(\",\", \"\")),
                except (AttributeError, ValueError):,
                    return None,
    return None,

def update_missing_production_cost(movie_df: pd.DataFrame) -> None:,
        "    \"\"\"Find missing production_cost values, scrape budget, and update the DataFrame in place.,
        ",
        "    :param movie_df: The DataFrame containing movie data.,
        "    \"\"\",
        "    missing_budget_rows = movie_df['production_cost'].isna(),
        ",
        "    for index in movie_df[missing_budget_rows].index:,
        "        imdb_id = movie_df.at[index, 'imdb_id'],
        "        budget = fetch_budget(imdb_id),
        ",
        "        if budget:,
        "            movie_df.at[index, 'production_cost'] = budget,
        "            print(f\"Updated {imdb_id} with budget: {budget}\"),
        "        else:,
        "            print(f\"No budget found for {imdb_id}\"),
        ",
        ",
        "def process_movie_data(file_path: str) -> None:,
        "    \"\"\"Reads, processes, updates, and saves the movie dataset.,
        ",
        "    :param file_path: Path to the input CSV file.,
        "    \"\"\",
        "    movie_df = pd.read_csv(file_path, delimiter=\";\"),
        ",
        "    # Update missing production costs directly in movie_df,
        "    update_missing_production_cost(movie_df),
        ",
        "    # Convert ratings to float,
        "    movie_df[\"average_rating\"] = movie_df[\"average_rating\"].str.replace(',', '.').astype(float),
        ",
        "    # Filter movies with a rating of 7 or higher,
        "    movie_df_filtered = movie_df[movie_df['average_rating'] >= 7][[\"imdb_id\", \"production_cost\"]],
        ",
        "    # Save updated files,
        "    movie_df.to_csv(\"movies_updated.csv\", index=False),
        "    movie_df_filtered.to_csv(\"movies_updated_filtered.csv\", index=False),
        "    print(\"Files saved successfully.\")